# NB for testing whole pipeline

In [17]:
%load_ext autoreload
%autoreload 2

import re

from loguru import logger

from ai_assistant.main import store_webpage
from ai_assistant.utils.db_utils import collection_cache
from ai_assistant.generator import init_gemini_pro
from ai_assistant.generator.qa import wiki_assistant

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Define some constants to facilitate testing

In [3]:
URL = "https://en.wikipedia.org/wiki/Bhagavad_Gita"
URL_LOCAL = URL.split("/")[-1]
URL_LOCAL = re.sub(r"\W+", "", URL_LOCAL)
CHROMA_COLLECTION_NAME = f"wiki_{URL_LOCAL}"
CHROMADB_DIR = "../db/"
UPDATE = False

## Index URL if not already indexed
Check if data from this wiki page already indexed or if update is enabled
If not following steps:
- Crawl data from wiki page
- Index to vector db collection
- Cache indexing infor to redis

In [5]:

conn = collection_cache()
logger.info("Check if data from this wiki page already indexed")
if not conn.check(URL) or UPDATE:
    logger.info(f"(re)indexing data from {URL}")
    store_webpage(URL, CHROMA_COLLECTION_NAME, CHROMADB_DIR)

2024-05-11 15:33:22.435 | INFO     | __main__:<module>:2 - Check if data from this wiki page already indexed


## Ask Question(s)

In [6]:
QUESTION = "What is the spiritual significance of the Bhagavad Gita?"

In [18]:
# Init Generator
gemini = init_gemini_pro(temperature=0.2)
assistant = wiki_assistant(
        retriver_collection=CHROMA_COLLECTION_NAME, database_loc=CHROMADB_DIR
    )
logger.info(f"Ask questions based on the wiki page about {URL_LOCAL}.")

2024-05-11 15:46:31.419 | INFO     | __main__:<module>:6 - Ask questions based on the wiki page about Bhagavad_Gita.


In [19]:
response = assistant(QUESTION)
print(f"Response: \n {response}")
print()

Response: 
 Context: [{'id': 'pid_106#0', 'score': 0.3875558964274768, 'long_text': 'with its translation and study by western scholars beginning in the early 18th century, the bhagavad gita gained a growing appreciation and popularity. [



In [20]:
gemini.inspect_history(1)




Given the fields `context`, `question`, produce the fields `answer`.

---

Follow the following format.

Context: ${context}

Question: ${question}

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: ${answer}

---

Context: [{'id': 'pid_106#0', 'score': 0.3875558964274768, 'long_text': 'with its translation and study by western scholars beginning in the early 18th century, the bhagavad gita gained a growing appreciation and popularity. [ web 1 ]', 'metadatas': {'source': 'https://en.wikipedia.org/wiki/Bhagavad_Gita', 'title': 'Reception'}}, {'id': 'pid_92#0', 'score': 0.45634493231773376, 'long_text': 'the bhagavad gita is referenced in the brahma sutras and numerous scholars wrote commentaries on it, including shankara, bhaskara, abhinavagupta, ramanuja, and madhvacharya. [ 310 ] [ 311 ] many of these commentators state that the gita is " meant to be a moksa - shastra ( moksasatra ), and not a dharmasastra, an arthasastra or a kamasastra ". [ 31

'\n\n\nGiven the fields `context`, `question`, produce the fields `answer`.\n\n---\n\nFollow the following format.\n\nContext: ${context}\n\nQuestion: ${question}\n\nReasoning: Let\'s think step by step in order to ${produce the answer}. We ...\n\nAnswer: ${answer}\n\n---\n\nContext: [{\'id\': \'pid_106#0\', \'score\': 0.3875558964274768, \'long_text\': \'with its translation and study by western scholars beginning in the early 18th century, the bhagavad gita gained a growing appreciation and popularity. [ web 1 ]\', \'metadatas\': {\'source\': \'https://en.wikipedia.org/wiki/Bhagavad_Gita\', \'title\': \'Reception\'}}, {\'id\': \'pid_92#0\', \'score\': 0.45634493231773376, \'long_text\': \'the bhagavad gita is referenced in the brahma sutras and numerous scholars wrote commentaries on it, including shankara, bhaskara, abhinavagupta, ramanuja, and madhvacharya. [ 310 ] [ 311 ] many of these commentators state that the gita is " meant to be a moksa - shastra ( moksasatra ), and not a dh